# Using PyTorch models with MarkovML

`markovml` supports PyTorch models that have been constructed using `torch.nn.Sequential`, with only ReLU and linear layers. In order to use a classifier, you must wrap the `Sequential` model in a `SequentialClassifier` class, which will indicate to `markovml` to also construct the piecewise-linear approximation for the softmax function.


## Setup

In [1]:
# Add the package to your Python path
import sys
sys.path.append("../")

from markovml.markovml import MarkovReward
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Sequential
from markovml.utils.models_ext import SequentialClassifier

A function to help us train a simple neural network. This code is not terribly important -- you would just be using your own training loop anyways.

In [2]:
def train_simple_nn(model, X, y, is_classifier=False, num_classes=None, epochs=10):
    optimizer = torch.optim.Adam(model.parameters())
    if is_classifier:
        if num_classes==2:
            criterion = nn.BCEWithLogitsLoss()
        else:
            criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.MSELoss()

    X_tensor = torch.FloatTensor(X)
    if is_classifier:
        y_tensor = torch.LongTensor(y) if num_classes>2 else torch.FloatTensor(y).view(-1, 1)
    else:
        y_tensor = torch.FloatTensor(y)

    for i in range(epochs):
        optimizer.zero_grad()
        output = model(X_tensor)
        loss = criterion(output, y_tensor)
        loss.backward()
        optimizer.step()

        print(f"Epoch {i+1} loss: {loss.item()}")

    model.eval()
    return model

## Training a simple neural network regressor

In [3]:
X = np.random.rand(1000, 2)  # 2 features for example
y = np.random.rand(1000).reshape(-1, 1)

# Create and train transition model
reg = Sequential(
    nn.Linear(2, 3),
    nn.ReLU(),
    nn.Linear(3, 1)
)
reg = train_simple_nn(reg, X, y, is_classifier=False, epochs=5)


Epoch 1 loss: 0.18397751450538635
Epoch 2 loss: 0.1828496903181076
Epoch 3 loss: 0.18173320591449738
Epoch 4 loss: 0.1806282103061676
Epoch 5 loss: 0.17953632771968842


## Training a simple neural network classifier

In [4]:
X = np.random.rand(1000, 2)  # 2 features for example
y = np.random.randint(0, 2, size=1000)

# Create and train transition model
clf = Sequential(
    nn.Linear(2, 3),
    nn.ReLU(),
    nn.Linear(3, 1)
)
clf = train_simple_nn(clf, X, y, is_classifier=True, num_classes=2, epochs=5)


Epoch 1 loss: 0.7392209768295288
Epoch 2 loss: 0.7388524413108826
Epoch 3 loss: 0.7384860515594482
Epoch 4 loss: 0.7381219267845154
Epoch 5 loss: 0.7377598881721497


## Integrating a neural network regressor into `markovml`

Let's first try integrating the regressor. We build a simple two-state Markov reward process and set one of the rewards equal to the output of the neural network regressor.

In [5]:
mrp = MarkovReward(n_states=2, n_features=2, pi=[0.5,0.5], P=[[0.5,0.5],[0.5,0.5]])
mrp.add_ml_model(reg)
mrp.set_r([5, mrp.ml_outputs[0][0]])

for f in mrp.features.values():
    f.LB = -1
    f.UB = 1

mrp.optimize()

Restricted license - for non-production use only - expires 2026-11-23


{'status': 'optimal',
 'objective': 87.49932597080857,
 'values': {'pi': [0.5, 0.5],
  'P': [[0.5, 0.5], [0.5, 0.5]],
  'r': [5.0, 0.2499595582485199],
  'v': [89.87434619168431, 85.12430574993284],
  'features': [0.5664458887997081, 0.44065540800513275],
  'ml_outputs': [[0.2499595582485199]]}}

## Integrating a neural network classifier into `markovml`

Now let's try building another Markov reward process but by integrating the classifier, and setting some probabilities to be functions of the output of the classifier. A *very important* thing to note is that the way we set up the classifier is that it does not have a softmax layer at the end. This is the common way to set up a classifier in PyTorch: use the cross-entropy loss (which only requires the logits, not probabilities), and then at inference time, apply the softmax function to the output. In order to get around this issue, we need to wrap our classifier in a `SequentialClassifier` class, which will add a softmax layer at the end. It also signals to `markovml` that this is a classifier and hence tells it to construct the piecewise-linear approximation for the softmax function and also applies appropriate bounds to the outputs. If you don't do this, it will not work properly!


Just to emphasize it again: if you intend to use a `Sequential` model as a classifier, **you must wrap it in a `SequentialClassifier` class**.

In [6]:
clf = SequentialClassifier(clf)

In [7]:
mrp = MarkovReward(n_states=2, n_features=2, pi=[0.5,0.5], r=[10, 5])
mrp.add_ml_model(clf)
mrp.set_P([[1 - mrp.ml_outputs[0][0], mrp.ml_outputs[0][0]], [0, 1]])

for f in mrp.features.values():
    f.LB = -1
    f.UB = 1

mrp.optimize()

{'status': 'optimal',
 'objective': 169.16674401365805,
 'values': {'pi': [0.5, 0.5],
  'P': [[0.0, 1.0000009568699972], [0.0, 1.0]],
  'r': [10.0, 5.0],
  'v': [171.6668213606494, 166.66666666666666],
  'features': [-1.0, -1.0],
  'ml_outputs': [[1.0000009568699972]]}}